In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
# !pip3 install tensorflow==1.15.5

In [3]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [4]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [5]:
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model.univnet import discriminator
from malaya_speech.train.model.univnet import model
from malaya_boilerplate.train import config
import tensorflow as tf

In [6]:
hparams = config.HParams(**malaya_speech.config.univnet_config_c16)

In [7]:
discriminator = discriminator.Discriminator(hparams)

In [8]:
generator = model.Generator(hparams)

In [9]:
MEL = tf.placeholder(tf.float32, (None, None, 80))
Y = tf.placeholder(tf.float32, (None, None))

In [10]:
z = tf.random.normal(shape = (tf.shape(MEL)[0], tf.shape(MEL)[1], 64))

In [11]:
y_ = generator(MEL, z = z)
y_

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


<tf.Tensor 'generator/sequential_1/activation/Tanh:0' shape=(?, ?, 1) dtype=float32>

In [12]:
from malaya_speech.train.model import stft
from malaya_boilerplate.train.loss import calculate_2d_loss, calculate_3d_loss

stft_loss = stft.loss.MultiResolutionSTFT()
sc_loss, mag_loss = calculate_2d_loss(
    Y, tf.squeeze(y_, -1), stft_loss
)

In [13]:
stft_loss = (sc_loss + mag_loss) * hparams.train.stft_lamb

In [14]:
res_fake, period_fake = discriminator(y_)

In [15]:
score_loss = 0.0
for (_, score_fake) in res_fake + period_fake:
    score_loss += tf.reduce_mean(tf.pow(score_fake - 1.0, 2))

In [16]:
score_loss = score_loss / len(res_fake + period_fake)
loss_g = score_loss + stft_loss

In [17]:
res_fake, period_fake = discriminator(tf.stop_gradient(y_))
res_real, period_real = discriminator(tf.expand_dims(Y, -1))

In [18]:
loss_d = 0.0
for (_, score_fake), (_, score_real) in zip(res_fake + period_fake, res_real + period_real):
    loss_d += tf.reduce_mean(tf.pow(score_real - 1.0, 2))
    loss_d += tf.reduce_mean(tf.pow(score_fake, 2))
    
loss_d = loss_d / len(res_fake + period_fake)

In [19]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [20]:
from glob import glob

audios = glob('../speech/example-speaker/*.wav')
audios

['../speech/example-speaker/haqkiem.wav',
 '../speech/example-speaker/husein-generated.wav',
 '../speech/example-speaker/khalil-nooh.wav',
 '../speech/example-speaker/muhyiddin-yassin.wav',
 '../speech/example-speaker/mas-aisyah.wav',
 '../speech/example-speaker/female.wav',
 '../speech/example-speaker/shafiqah-idayu.wav',
 '../speech/example-speaker/husein-zolkepli.wav']

In [28]:
audio, _ = malaya_speech.load(audios[0], sr = 22050)
audio.shape

(181504,)

In [29]:
from malaya_speech.utils.featurization import universal_mel

mel = universal_mel(audio)
mel.shape

(710, 80)

In [30]:
out_ = sess.run(y_, feed_dict = {MEL: [mel]})
out_.shape

(1, 181760, 1)

In [26]:
import random

batch_max_steps=8192
hop_size=256

batch_max_frames = batch_max_steps // hop_size
if len(audio) < len(mel) * hop_size:
    audio = np.pad(audio, [[0, len(mel) * hop_size - len(audio)]])

if len(mel) > batch_max_frames:
    interval_start = 0
    interval_end = len(mel) - batch_max_frames
    start_frame = random.randint(interval_start, interval_end)
    start_step = start_frame * hop_size
    audio = audio[start_step: start_step + batch_max_steps]
    mel = mel[start_frame: start_frame + batch_max_frames, :]
else:
    audio = np.pad(audio, [[0, batch_max_steps - len(audio)]])
    mel = np.pad(mel, [[0, batch_max_frames - len(mel)], [0, 0]])

In [27]:
audio.shape, mel.shape

((8192,), (32, 80))

In [31]:
loss_g

<tf.Tensor 'add_9:0' shape=(?,) dtype=float32>

In [35]:
out_ = sess.run(loss_g, feed_dict = {MEL: [mel], Y: [audio]})
out_

array([25.581398], dtype=float32)

In [34]:
out_ = sess.run(loss_d, feed_dict = {MEL: [mel], Y: [audio]})
out_

1.4679272